In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
# # install octave
# !sudo apt-get -qq update
# !sudo apt-get -qq install octave octave-signal liboctave-dev

# # install oct2py that compatible with colab
# import os

# from pkg_resources import get_distribution

# os.system(
#     f"pip install -qq"
#     f" ipykernel=={get_distribution('ipykernel').version}"
#     f" ipython=={get_distribution('ipython').version}"
#     f" tornado=={get_distribution('tornado').version}"
#     f" oct2py"
# )

# # install packages
# !pip install -qq matpower matpowercaseframes

In [3]:
import matpower
import oct2py

print(f"oct2py version: {oct2py.__version__}")
print(f"matpower version: {matpower.__version__}")

oct2py version: 5.8.0
matpower version: 8.1.0.2.2.3


In [22]:
import os

import pandas as pd
from matpower import path_matpower, start_instance

from matpowercaseframes import CaseFrames, xGenDataTableFrames
from matpowercaseframes.constants import ATTRIBUTES_INFO

In [5]:
path_most_ex_cases = os.path.join(path_matpower, "most/examples")

In [6]:
m = start_instance()

In [7]:
verbose = 1
mpopt = m.mpoption("verbose", verbose)
mpopt = m.mpoption(mpopt, "out.gen", 1)
mpopt = m.mpoption(mpopt, "model", "DC")
mpopt = m.mpoption(mpopt, "opf.dc.solver", "MIPS")
mpopt = m.mpoption(mpopt, "most.solver", mpopt.opf.dc.solver)
mpopt = m.mpoption(mpopt, "most.dc_model", 0)  # use model with no network
mpopt

{'v': 25.0,
 'model': 'DC',
 'pf': {'alg': 'NR',
  'current_balance': 0.0,
  'v_cartesian': 0.0,
  'tol': 1e-08,
  'nr': {'max_it': 10.0, 'lin_solver': ''},
  'fd': {'max_it': 30.0},
  'gs': {'max_it': 1000.0},
  'zg': {'max_it': 1000.0},
  'radial': {'max_it': 20.0, 'vcorr': 0.0},
  'enforce_q_lims': 0.0},
 'cpf': {'parameterization': 3.0,
  'stop_at': 'NOSE',
  'enforce_p_lims': 0.0,
  'enforce_q_lims': 0.0,
  'enforce_v_lims': 0.0,
  'enforce_flow_lims': 0.0,
  'step': 0.05,
  'step_min': 0.0001,
  'step_max': 0.2,
  'adapt_step': 0.0,
  'adapt_step_damping': 0.7,
  'adapt_step_tol': 0.001,
  'target_lam_tol': 1e-05,
  'nose_tol': 1e-05,
  'p_lims_tol': 0.01,
  'q_lims_tol': 0.01,
  'v_lims_tol': 0.0001,
  'flow_lims_tol': 0.01,
  'plot': {'level': 0.0, 'bus': []},
  'user_callback': ''},
 'opf': {'ac': {'solver': 'DEFAULT'},
  'dc': {'solver': 'MIPS'},
  'current_balance': 0.0,
  'v_cartesian': 0.0,
  'violation': 5e-06,
  'use_vg': 0.0,
  'flow_lim': 'S',
  'ignore_angle_lim': 0.0

## Example 1 - Economic Dispatch

In [23]:
def display_cf(cf):
    data = {"INFO": {}}
    for attribute in ATTRIBUTES_INFO:
        if attribute in cf._attributes:
            data["INFO"][attribute] = getattr(cf, attribute, None)
    display(pd.DataFrame(data=data))

    for attribute in cf._attributes:
        if attribute in ATTRIBUTES_INFO or attribute == "reserves":
            continue
        print(attribute)
        display(getattr(cf, attribute))

    for attribute in cf.reserves._attributes:
        print(attribute)
        display(getattr(cf.reserves, attribute))

In [24]:
cf = CaseFrames(os.path.join(path_most_ex_cases, "ex_case3a.m"), load_case_engine=m)
display_cf(cf)

,INFO
version,2
baseMVA,100.0


bus


,BUS_I,BUS_TYPE,PD,QD,GS,BS,BUS_AREA,VM,VA,BASE_KV,ZONE,VMAX,VMIN
bus,,,,,,,,,,,,,
1,1.0,3.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,135.0,1.0,1.05,0.95
2,2.0,2.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,135.0,1.0,1.05,0.95
3,3.0,2.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,135.0,1.0,1.05,0.95


gen


,GEN_BUS,PG,QG,QMAX,QMIN,VG,MBASE,GEN_STATUS,PMAX,PMIN,...,PC2,QC1MIN,QC1MAX,QC2MIN,QC2MAX,RAMP_AGC,RAMP_10,RAMP_30,RAMP_Q,APF
gen,,,,,,,,,,,,,,,,,,,,,
1,1.0,125.0,0.0,25.0,-25.0,1.0,100.0,1.0,200.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,250.0,250.0,0.0,0.0
2,1.0,125.0,0.0,25.0,-25.0,1.0,100.0,1.0,200.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,250.0,250.0,0.0,0.0
3,2.0,200.0,0.0,50.0,-50.0,1.0,100.0,1.0,500.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,600.0,600.0,0.0,0.0
4,3.0,-450.0,0.0,0.0,0.0,1.0,100.0,1.0,0.0,-450.0,...,0.0,0.0,0.0,0.0,0.0,0.0,500.0,500.0,0.0,0.0


...

,PREQ
zone,
1,150.0


cost


,C1
gen,
1,1.0
2,3.0
3,5.0


qty


,PQTY
gen,
1,100.0
2,100.0
3,200.0


In [9]:
# TODO: convert mdi into DataFrame
# mdi: MOST data structure input
mdi = m.loadmd(cf.to_mpc())
mdi

{'Delta_T': 1.0,
 'Storage': {'UnitIdx': [],
  'ExpectedTerminalStorageAim': [],
  'ExpectedTerminalStorageMin': [],
  'ExpectedTerminalStorageMax': [],
  'rho': [],
  'TerminalChargingPrice0': [],
  'TerminalDischargingPrice0': [],
  'TerminalChargingPriceK': [],
  'TerminalDischargingPriceK': []},
 'UC': {'CommitKey': [],
  'CommitSched': array([[1.],
         [1.],
         [1.],
         [1.]])},
 'idx': {'nt': 1.0},
 'tstep': {'TransMat': 1.0, 'OpCondSched': {'tab': []}},
 'mpc': {'version': '2',
  'baseMVA': 100.0,
  'bus': array([[  1.  ,   3.  ,   0.  ,   0.  ,   0.  ,   0.  ,   1.  ,   1.  ,
            0.  , 135.  ,   1.  ,   1.05,   0.95],
         [  2.  ,   2.  ,   0.  ,   0.  ,   0.  ,   0.  ,   1.  ,   1.  ,
            0.  , 135.  ,   1.  ,   1.05,   0.95],
         [  3.  ,   2.  ,   0.  ,   0.  ,   0.  ,   0.  ,   1.  ,   1.  ,
            0.  , 135.  ,   1.  ,   1.05,   0.95]]),
  'gen': array([[   1.,  125.,    0.,   25.,  -25.,    1.,  100.,    1.,  200.,
         

In [10]:
# TODO:
#   1. Convert mdo into DataFrame
#   2. Clean up classdef element has been converted to a struct
# mdo: MOST data structure output
mdo = m.most(mdi, mpopt)
mdo


          MATPOWER Optimal Scheduling Tool  --  MOST Version 1.3.1
          A multiperiod stochastic secure OPF with unit commitment
                       -----  Built on MATPOWER  -----
  by Carlos E. Murillo-Sanchez, Universidad Nacional de Colombia--Manizales
                  and Ray D. Zimmerman, Cornell University
       (c) 2010-2025 Power Systems Engineering Research Center (PSERC)
- Building indexing structures.
- Building expected storage-tracking mechanism.
- Building constraint submatrices.
  - Building load balance constraints.
  - Building CCV constraints for piecewise-linear costs.
  - Building contingency reserve constraints.
  - Building ramping transitions and reserve constraints.
- Building cost structures.
- Assembling full set of constraints.
- Assembling full set of variable bounds.
- Assembling full set of costs.
- Calling QP solver.


MATPOWER Interior Point Solver -- MIPS, Version 1.5.2, 12-Jul-2025
 (using built-in linear solver)
Converged!

- MOST: QP solv

{'Delta_T': 1.0,
 'Storage': {'UnitIdx': [],
  'ExpectedTerminalStorageAim': [],
  'ExpectedTerminalStorageMin': [],
  'ExpectedTerminalStorageMax': [],
  'rho': [],
  'TerminalChargingPrice0': [],
  'TerminalDischargingPrice0': [],
  'TerminalChargingPriceK': [],
  'TerminalDischargingPriceK': [],
  'ForceCyclicStorage': 0.0,
  'ForceExpectedTerminalStorage': 0.0,
  'ExpectedStorageState': array([], shape=(0, 1), dtype=float64)},
 'UC': {'CommitKey': [],
  'CommitSched': array([[1.],
         [1.],
         [1.],
         [1.]]),
  'run': 0.0,
  'CyclicCommitment': 0.0},
 'idx': {'nt': 1.0,
  'ng': 4.0,
  'ns': 0.0,
  'nj': 1.0,
  'ntds': 0.0,
  'nzds': 0.0,
  'nyds': 0.0,
  'nb': 3.0,
  'ny': 0.0,
  'nc': 0.0,
  'nf_total': 1.0,
  'nb_total': 3.0,
  'ns_total': 0.0,
  'ntramp': 0.0,
  'nvars': 24.0},
 'tstep': {'TransMat': 1.0, 'OpCondSched': {'tab': []}, 'TransMask': 1.0},
 'mpc': {'version': '2',
  'baseMVA': 100.0,
  'bus': array([[  1.  ,   3.  ,   0.  ,   0.  ,   0.  ,   0.  ,  

In [11]:
# TODO:
#   1. Fix error
#       Oct2PyError: Octave evaluation error:
#       error: get: H must be a graphics handle
#       error: called from:
#   2. ms is most summary

# ms = m.most_summary(mdo)
# ms

In [25]:
cfr2 = CaseFrames(mdo.flow.mpc)
display_cf(cfr2)

,INFO
version,2
baseMVA,100.0
f,0.0
et,0.0
success,1.0


bus


,BUS_I,BUS_TYPE,PD,QD,GS,BS,BUS_AREA,VM,VA,BASE_KV,ZONE,VMAX,VMIN,LAM_P,LAM_Q,MU_VMAX,MU_VMIN
bus,,,,,,,,,,,,,,,,,
1,1.0,3.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,135.0,1.0,1.05,0.95,30.0,0.0,0.0,0.0
2,2.0,2.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,135.0,1.0,1.05,0.95,30.0,0.0,0.0,0.0
3,3.0,2.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,135.0,1.0,1.05,0.95,30.0,0.0,0.0,0.0


gen


,GEN_BUS,PG,QG,QMAX,QMIN,VG,MBASE,GEN_STATUS,PMAX,PMIN,...,QC2MAX,RAMP_AGC,RAMP_10,RAMP_30,RAMP_Q,APF,MU_PMAX,MU_PMIN,MU_QMAX,MU_QMIN
gen,,,,,,,,,,,,,,,,,,,,,
1,1.0,150.0,0.0,25.0,-25.0,1.0,100.0,1.0,200.0,0.0,...,0.0,0.0,250.0,250.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1.0,150.0,0.0,25.0,-25.0,1.0,100.0,1.0,200.0,0.0,...,0.0,0.0,250.0,250.0,0.0,0.0,0.0,0.0,0.0,0.0
3,2.0,150.0,0.0,50.0,-50.0,1.0,100.0,1.0,500.0,0.0,...,0.0,0.0,600.0,600.0,0.0,0.0,0.0,0.0,0.0,0.0
4,3.0,-450.0,0.0,0.0,0.0,1.0,100.0,1.0,0.0,-450.0,...,0.0,0.0,500.0,500.0,0.0,0.0,0.0,970.0,0.0,0.0


...

,PREQ
zone,
1,150.0


cost


,C1
gen,
1,1.0
2,3.0
3,5.0


qty


,PQTY
gen,
1,100.0
2,100.0
3,200.0


In [27]:
Pg2 = cfr2.gen.loc[:, "PG"]  # active generation
lam2 = cfr2.bus.loc[:, "LAM_P"]  # nodal energy price
Pg2, lam2

(gen
 1    150.0
 2    150.0
 3    150.0
 4   -450.0
 Name: PG, dtype: float64,
 bus
 1    30.0
 2    30.0
 3    30.0
 Name: LAM_P, dtype: float64)

> [!NOTE]  
> Currently, `loadgenericdata` didn't support absolute path. This impacts various `most`
> functions. See: <https://github.com/MATPOWER/matpower/pull/283>.

In [80]:
# NOTE:
#   This code below and `m.loadxgendata(xgdtf.to_dict(), mpc)` can be skipped using,
# `xgd = m.loadxgendata("ex_xgd_uc.m", mpc)`
xgdt = m.loadgenericdata(
    "ex_xgd_uc.m", "struct", {"colnames", "data"}, "xgd_table", cf.to_mpc()
)
xgdtf = xGenDataTableFrames(data=xgdt.data, colnames=xgdt.colnames, index=cf.gen.index)
xgdtf

,CommitKey,CommitSched,MinUp,MinDown,PositiveActiveReservePrice,PositiveActiveReserveQuantity,NegativeActiveReservePrice,NegativeActiveReserveQuantity,PositiveActiveDeltaPrice,NegativeActiveDeltaPrice,PositiveLoadFollowReservePrice,PositiveLoadFollowReserveQuantity,NegativeLoadFollowReservePrice,NegativeLoadFollowReserveQuantity
gen,,,,,,,,,,,,,,
1,1.0,1.0,1.0,1.0,5.000000e+00,250.0,1.000000e+01,250.0,1.000000e-09,1.000000e-09,0.000001,250.0,0.000001,250.0
2,1.0,1.0,3.0,1.0,1.000000e-08,250.0,2.000000e-08,250.0,1.000000e-09,1.000000e-09,0.000001,250.0,0.000001,250.0
3,1.0,1.0,1.0,1.0,1.500000e+00,600.0,3.000000e+00,600.0,1.000000e-09,1.000000e-09,10.000000,100.0,10.000000,250.0
4,2.0,1.0,1.0,1.0,1.000000e-08,800.0,2.000000e-08,800.0,1.000000e-09,1.000000e-09,0.000001,800.0,0.000001,800.0


In [81]:
xgd = m.loadxgendata(xgdtf.to_xgdt(), cf.to_mpc())
xgdf = xGenDataTableFrames(data=xgd)
xgdf  # support both `xgdf["CommitSched"]` and `xgdf.CommitSched`

,CommitSched,InitialPg,RampWearCostCoeff,PositiveActiveReservePrice,PositiveActiveReserveQuantity,NegativeActiveReservePrice,NegativeActiveReserveQuantity,PositiveActiveDeltaPrice,NegativeActiveDeltaPrice,PositiveLoadFollowReservePrice,PositiveLoadFollowReserveQuantity,NegativeLoadFollowReservePrice,NegativeLoadFollowReserveQuantity,CommitKey,InitialState,MinUp,MinDown
gen,,,,,,,,,,,,,,,,,
1,1.0,125.0,0.0,5.000000e+00,250.0,1.000000e+01,250.0,1.000000e-09,1.000000e-09,0.000001,250.0,0.000001,250.0,1.0,inf,1.0,1.0
2,1.0,125.0,0.0,1.000000e-08,250.0,2.000000e-08,250.0,1.000000e-09,1.000000e-09,0.000001,250.0,0.000001,250.0,1.0,inf,3.0,1.0
3,1.0,200.0,0.0,1.500000e+00,600.0,3.000000e+00,600.0,1.000000e-09,1.000000e-09,10.000000,100.0,10.000000,250.0,1.0,inf,1.0,1.0
4,1.0,-450.0,0.0,1.000000e-08,800.0,2.000000e-08,800.0,1.000000e-09,1.000000e-09,0.000001,800.0,0.000001,800.0,2.0,inf,1.0,1.0


In [ ]:
# NOTE:
#   1. iwind, the wind index in cf.gen, can be singleton or 2D array of (n_wind, 1)
#   2. mpc and xgd are updated with wind generators added
[iwind, mpc, xgd] = m.addwind("ex_wind_uc", cf.to_mpc(), xgdf.to_xgd(), nout=3)
xgdf = xGenDataTableFrames(data=xgd)
cf = CaseFrames(mpc)

display(xgdf)
display(cf.gen)

,CommitSched,InitialPg,RampWearCostCoeff,PositiveActiveReservePrice,PositiveActiveReserveQuantity,NegativeActiveReservePrice,NegativeActiveReserveQuantity,PositiveActiveDeltaPrice,NegativeActiveDeltaPrice,PositiveLoadFollowReservePrice,PositiveLoadFollowReserveQuantity,NegativeLoadFollowReservePrice,NegativeLoadFollowReserveQuantity,CommitKey,InitialState,MinUp,MinDown
gen,,,,,,,,,,,,,,,,,
1,1.0,125.0,0.0,5.000000e+00,250.0,1.000000e+01,250.0,1.000000e-09,1.000000e-09,0.000001,250.0,0.000001,250.0,1.0,inf,1.0,1.0
2,1.0,125.0,0.0,1.000000e-08,250.0,2.000000e-08,250.0,1.000000e-09,1.000000e-09,0.000001,250.0,0.000001,250.0,1.0,inf,3.0,1.0
3,1.0,200.0,0.0,1.500000e+00,600.0,3.000000e+00,600.0,1.000000e-09,1.000000e-09,10.000000,100.0,10.000000,250.0,1.0,inf,1.0,1.0
4,1.0,-450.0,0.0,1.000000e-08,800.0,2.000000e-08,800.0,1.000000e-09,1.000000e-09,0.000001,800.0,0.000001,800.0,2.0,inf,1.0,1.0
5,1.0,0.0,0.0,1.000000e-08,200.0,2.000000e-08,200.0,1.000000e-09,1.000000e-09,0.000001,200.0,0.000001,200.0,2.0,inf,1.0,1.0
6,1.0,0.0,0.0,1.000000e-08,200.0,2.000000e-08,200.0,1.000000e-09,1.000000e-09,0.000001,200.0,0.000001,200.0,2.0,inf,1.0,1.0


,GEN_BUS,PG,QG,QMAX,QMIN,VG,MBASE,GEN_STATUS,PMAX,PMIN,...,PC2,QC1MIN,QC1MAX,QC2MIN,QC2MAX,RAMP_AGC,RAMP_10,RAMP_30,RAMP_Q,APF
gen,,,,,,,,,,,,,,,,,,,,,
1,1.0,125.0,0.0,25.0,-25.0,1.0,100.0,1.0,200.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,250.0,250.0,0.0,0.0
2,1.0,125.0,0.0,25.0,-25.0,1.0,100.0,1.0,200.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,250.0,250.0,0.0,0.0
3,2.0,200.0,0.0,50.0,-50.0,1.0,100.0,1.0,500.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,600.0,600.0,0.0,0.0
4,3.0,-450.0,0.0,0.0,0.0,1.0,100.0,1.0,0.0,-450.0,...,0.0,0.0,0.0,0.0,0.0,0.0,500.0,500.0,0.0,0.0
5,2.0,0.0,0.0,50.0,-50.0,1.0,100.0,1.0,100.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,200.0,200.0,0.0,0.0
6,2.0,0.0,0.0,50.0,-50.0,1.0,100.0,1.0,100.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,200.0,200.0,0.0,0.0


In [ ]:
# NOTE:
#   1. iess, the ess index in cf.gen, can be singleton or 2D array of (n_ess, 1)
#   2. mpc and xgd are updated with ess added
[iess, mpc, xgd, sd] = m.addstorage("ex_storage", mpc, xgd, nout=4)
xgdf = xGenDataTableFrames(data=xgd)
cf = CaseFrames(mpc)
# TODO: convert sd into DataFrame

display(xgdf)
display(cf.gen)

,CommitSched,InitialPg,RampWearCostCoeff,PositiveActiveReservePrice,PositiveActiveReserveQuantity,NegativeActiveReservePrice,NegativeActiveReserveQuantity,PositiveActiveDeltaPrice,NegativeActiveDeltaPrice,PositiveLoadFollowReservePrice,PositiveLoadFollowReserveQuantity,NegativeLoadFollowReservePrice,NegativeLoadFollowReserveQuantity,CommitKey,InitialState,MinUp,MinDown
gen,,,,,,,,,,,,,,,,,
1,1.0,125.0,0.0,5.000000e+00,250.0,1.000000e+01,250.0,1.000000e-09,1.000000e-09,0.000001,250.0,0.000001,250.0,1.0,inf,1.0,1.0
2,1.0,125.0,0.0,1.000000e-08,250.0,2.000000e-08,250.0,1.000000e-09,1.000000e-09,0.000001,250.0,0.000001,250.0,1.0,inf,3.0,1.0
3,1.0,200.0,0.0,1.500000e+00,600.0,3.000000e+00,600.0,1.000000e-09,1.000000e-09,10.000000,100.0,10.000000,250.0,1.0,inf,1.0,1.0
4,1.0,-450.0,0.0,1.000000e-08,800.0,2.000000e-08,800.0,1.000000e-09,1.000000e-09,0.000001,800.0,0.000001,800.0,2.0,inf,1.0,1.0
5,1.0,0.0,0.0,1.000000e-08,200.0,2.000000e-08,200.0,1.000000e-09,1.000000e-09,0.000001,200.0,0.000001,200.0,2.0,inf,1.0,1.0


,GEN_BUS,PG,QG,QMAX,QMIN,VG,MBASE,GEN_STATUS,PMAX,PMIN,...,PC2,QC1MIN,QC1MAX,QC2MIN,QC2MAX,RAMP_AGC,RAMP_10,RAMP_30,RAMP_Q,APF
gen,,,,,,,,,,,,,,,,,,,,,
1,1.0,125.0,0.0,25.0,-25.0,1.0,100.0,1.0,200.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,250.0,250.0,0.0,0.0
2,1.0,125.0,0.0,25.0,-25.0,1.0,100.0,1.0,200.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,250.0,250.0,0.0,0.0
3,2.0,200.0,0.0,50.0,-50.0,1.0,100.0,1.0,500.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,600.0,600.0,0.0,0.0
4,3.0,-450.0,0.0,0.0,0.0,1.0,100.0,1.0,0.0,-450.0,...,0.0,0.0,0.0,0.0,0.0,0.0,500.0,500.0,0.0,0.0
5,2.0,0.0,0.0,50.0,-50.0,1.0,100.0,1.0,100.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,200.0,200.0,0.0,0.0


In [ ]:
# TODO: convert profiles into DataFrame
profiles = m.getprofiles("ex_wind_profile_d", iwind)  # add wind profiles
profiles = m.getprofiles("ex_load_profile", profiles)  # update profiles with load
wind_profile, load_profile = profiles[0], profiles[1]

In [14]:
# m.exit()